## A large portion of this code is taken from Aurélien Géron's: Hands-On machine learning with SciKit-Learn, Keras and Tensorflow (2nd edition). I have put comments based on information in the book as well as information I found elsewhere.

## Chapter 11

## Vanishing/Exploding gradients problem

Vanishing gradients:
  - If the gradients at the upper layer are small, they're divided and assigned to each neuron in the lower layer.
    These values are then further divided and assigned to each neron in the layer below it.
    As the number of layers are large in a Deep Neural Network, the gradients at the bottom are zero.
    During forward propagation, this causes the weights to remain the same and training never converges.
Exploding gradients:
  - If the loss function has a steep gradient at the location where you're computing it,
    there will be a large change in the weight values. This in turn can cause you to overshoot the minima
    and land on the other side of the loss function. If the gradient at that point is also large,
    this can cause another large change in the weight values, and another overshoot.
  - Sometimes these overshoots will get larger and larger getting you more and more away from the minima.
    Sometimes these overshoots will get you into a region of the loss function where the gradients are smaller,
    thus temporarily keeping the changes to the weights low. But if you get back down to where the
    gradients are large, you may find yourself again overshooting. This will cause you to oscillate,
    and never get to the minima.
  - More generally, neural networks suffer from unstable gradients. 
    Different layers may learn at widely different speeds.

These problems result from using the Sigmoid activation function, 
and the Standard Normal initial weight distribution
(0 mean, 1 std. dev.). With this initialization, 
the input variance for each layer increases at the output of that layer. 
This causes the top layer's sigmoid function to get input that saturates it's output.
With a saturated output, the gradient is close to zero.
Backpropagation has no gradient to propagate back,
any small gradient keeps getting diluted at each lower layer.
The Sigmoid function has a mean of 0.5, which exacerbates the problem.
The Hyperbolic tangent function behaves better (since it's mean is 0).

### Glorot and He Initialization

Solution is to use Glorot initialization:
$$fan_{avg} = \frac{fan_{in} + fan_{out}}{2}$$
Initialize weights with Normal distribution with 0 mean, variance $\sigma^2 = \frac{1}{fan_{avg}}$

Or, a uniform distribution between $\pm$r, with $r = \sqrt{\frac{3}{fan_{avg}}} = \sqrt{3{\sigma}^2}$

Glorot initialization can speed up training considerably.

| Initialization | Activation functions | $\sigma^2$ (Normal) |
|---|---|---|
|Glorot|None, tanh, logistic, softmax|$\frac{1}{fan_{avg}}$|
|He|ReLU and variants|$\frac{2}{fan_{in}}$|
|LeCun|SELU|$\frac{1}{fan_{in}}$|

By default Keras uses Glorot initialization with a uniform distribution.
To use He init with $fan_{in}$:
  - keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal') # or kernel_initializer='he_uniform'

To use He init with $fan_{avg}$:
  - he_avg_init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution='uniform')
  - keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)
  - With distribution="uniform", samples are drawn from a uniform distribution within [-limit, limit], with 
    - limit = sqrt(3 * scale / n).
    - n = number of input weights
  - So you can specify the limits of the uniform distribution by setting the scale value

In [2]:
# By default Keras uses Glorot initialization with a uniform distribution. To use He init with 𝑓𝑎𝑛_𝑖𝑛:

# keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal') 
# or kernel_initializer='he_uniform'

# To use He init with 𝑓𝑎𝑛_𝑎𝑣𝑔:
# he_avg_init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution='uniform')
# keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

# With distribution="uniform", samples are drawn from a uniform distribution within [-limit, limit], with
#     limit = sqrt(3 * scale / n).
#     n = number of input weights
# So you can specify the limits of the uniform distribution by setting the scale value

### Nonsaturating Activation Functions

ReLU:
  - $$\begin{align}RELU(z) & = 0 \;\;\; if \; z < 0 \\
                       & = z \;\;\; if \; z >= 0 \end{align}$$                   
  - ReLU function thus does not saturate for positive values.
  - It is also fast to compute, since we only have to look at
    z to decide the output.
  - If a layer's weighted sums are negative for all input instances,
    the output is 0 (and stays 0 forever). This is the dying ReLU problem.
  - Use Leaky ReLU instead
  
Leaky ReLU:
  - $$Leaky \; ReLU_\alpha(z) = max(\alpha z, z)$$
  - $\alpha$ is the slope of the function for negative z
  - Setting $\alpha$ to 0.2 (a huge leak) seems to perform better than 0.01 (a small leak)
  - Default value of $\alpha$ = 0.3
  - **Preferred if you're worried about runtime latency**
  
RReLU (Randomized Leaky ReLU) 
  - uses random $\alpha$ for training, 
    and a fixed average $\alpha$ for testing.
    It performed fairly well and seemed to act as a regularizer
  - **Preferred if you're overfitting**
  
PReLU (Parametric Leaky ReLU) 
  - allows $\alpha$ to be learned during training.
    It becomes a parameter that backprop can modify.
    It strongly outperformed ReLU on large datasets, but risks overfitting on smaller datasets.
  - **Preferred if you have a huge training set**
  
ELU (Exponential Linear Unit):
  - Outperformed all ReLUs
  - Training time was reduced
  - Network performed better on test set
  - $$\begin{align}ELU_\alpha(z) & = \alpha(e^z - 1) & if \; z < 0 \\
                                 & = z & if \; z >= 0 \end{align}$$
  - ELU increases linearly with slope 1 for z > 0, and
    becomes more and more negative with an asymptote at -$\alpha$ as z becomes more negative
  - If $\alpha\,=\,1$, the function is smooth everywhere, including at z = 0.
    This allows Gradient Descent to speed up, as it does not bounce left and right of 0.
  - ELU is slower to compute. It's faster training rate helps at training time,
    but slow computation hurts during testing.
    
SELU (Scaled ELU):
  - If:
    - Your network consists only of a stack on Dense layers
    - All hidden layers use the SELU activation function
    - Input features are all standardized (mean 0, std. dev. 1)
    - All hidden-layer weights are initialized with LeCun normalization (kernel_initializer = 'lecun_normal')
    - Network's architecture must be sequential. No Recurrent Networks, Skip connections (Wide and Deep nets).
    - Some researchers have mentioned SELU works well for Convolutional Networks as well (even though there are no Dense layers).
    
**In general, prefer SELU > ELU > Leaky ReLU (and it's variants) > tanh > logistic**

In [3]:
# This is how you use the ReLU/SELU functions
#
# model = keras.models.Sequential([
#     [ ... ]
#     keras.layers.Dense(10, kernel_initializer = "he_normal"), 
#     keras.layers.LeakyReLU(alpha = 0.2),   # For LeakyReLU, this layer should come after each layer you want to
#                                            # apply it to. Default alpha = 0.3.
#                                            # For PReLU, replace this LeakyReLU() with PReLU().
#                                            # No implementation of RReLU() yet - you write your own.
#     [ ... ]
# ]) 

# layer = keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal')

## Batch Normalization (BN)

Why do we need it:
  - The Glorot/He initialization plus any nonsaturating activation function reduces the danger of vanishing/exploding gradients at the beginning of training. But they can come back during training. BN addresses this issue.

How:
  - Add BN layer before/after the activation function.

How does it work:
  - Evaluates mean and std dev of the input over the current mini-batch (hence it's called Batch Normalization).
  
  $$\displaystyle\begin{align}\mu_B & = \frac{1}{m_B}\sum_{i=1}^{m_B}x^{(i)} \\
                          {\sigma_B}^2 & = \frac{1}{m_B}\sum_{i=1}^{m_B}{(x^{(i)} - \mu_B)}^2\end{align}$$
  - 0-centers and normalizes each input, then scales and shifts it according to two new parameter vectors per layer.
  
  $$\begin{align}\widehat{x}^{(i)} & = \frac{x^{(i)}\;-\;\mu_B}{\sqrt{{\sigma_B}^2\;+\;\epsilon}} \\
                           z^{(i)} & = \gamma\;\otimes\;\widehat{x}^{(i)}\;+\;\beta\end{align}$$
                           
  <paragraph><center>where $\;\otimes \;=\;$ elementwise$\;$ multiplication</center></paragraph>
  
  <paragraph><center>$\epsilon \;=\; $small number to avoid divide-by-zero error, called a smoothing term, typically $10^{-5}$</center></paragraph>
  - During backpropagation, each batch-normalized layer learns:
    - $\gamma$ : the output scale vector
    - $\beta$  : the output offset vector
    - $\mu$    : final input mean vector (learned by using a moving average of the layer's input mean)
    - $\sigma$ : final input std dev vector (learned by using a moving average of the layer's input std dev)
  - Benefits:
    - Improved neural networks
    - Strongly reduced vanishing gradients problem
    - Networks were also much less sensitive to weight initialization
    - Could use much larger learning rates, speeding up learning
    - Acts as a regularizer, reducing the need for anu other regulatization technique
    - Does not affect shallower networks as much, but has a tremendous impact on deep networks.
    - Although converges faster, training per epoch is rather slow. All in all, wall time will be shorter.

In [ ]:
# AAdd a Batch Normalization layer before/after each hidden layer's activation function,
# and optionally after the first layer in your model (as shown here).
#
# model = Sequential([  # In this tiny example with just 2 hidden layers, it's unlikely that Batch Normalization
#                       # will have a very positive impact. For deeper networks, it can make a tremendous difference.
#   Flatten(input_shape=[28, 28]),
#   BatchNormalization(),
#   Dense(300, activation='elu', kernel_initializer='he_normal'),
#   BatchNormalization(),
#   Dense(100, activation='elu', kernel_initializer='he_normal'),
#   BatchNormalization(),
#   Dense(10, activation='softmax')
# ])
# model.summary()  # This will show you the Batch Normalization layers. The number of parameters
#                  # added for the BN layers is 4 x (number of input parameters).
#                  # mu and sigma are not trainable via backprop, and this is what Keras shows at the bottom.
# [(var.name, var.trainable) for var in model.layers[1].variables] # will list trainable/untrainable parameters.
#
# model.layers[1].updates  # Shows the operations Keras created to train the trainable params (gamma, beta)
#                          # at each iteration. Since we're using the Tensorflow backend, these are TF operations.

# Shows how to add BN layers before the activation function.
# You should try adding them before and after the activation function,
# and choosing the way that works best.
# model = Sequential([
#   Flatten(input_shape=[28, 28]),
#   BatchNormalization(),
#   Dense(300, kernel_initializer='he_normal', use_bias=False), # Removed activation function.
#                                                               # Since BN layer already has bias,
#                                                               # remove bias from the layer using use_bias=False.
#   BatchNormalization(),
#   Activation('elu'),
#   Dense(100, kernel_initializer='he_normal', use_bias=False),
#   BatchNormalization(),
#   Activation('elu'),
#   Dense(10, activation='softmax')
# ])
# 

In [ ]:
# BN layer has a lot of hyperparameters you can tweak.
# Most defaults will be fine.
# Usually you would need to tweak the momentum.